# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
df = pd.read_csv('1950-2021_fires_with_elevation.csv')
x = df[['lat', 'lon']]
y = df['hectares']

In [6]:
x

,lat,lon
0,59.963,-128.172
1,59.318,-132.172
2,59.876,-131.922
3,59.760,-132.808
4,59.434,-126.172
...,...,...
15025,58.818,-122.812
15026,58.289,-121.729
15027,58.818,-122.729
15028,58.760,-122.729


In [9]:
y

0            8.0
1            8.0
2        12949.9
3          241.1
4            1.2
          ...   
15025        0.1
15026        0.1
15027        0.1
15028        1.2
15029        0.1
Name: hectares, Length: 15030, dtype: float64

In [10]:
df.hectares.corr(df.elevation)

np.float64(0.006771238508371042)

# Test Train Split

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)